In [6]:
import mlflow
import os

In [27]:
! mlflow --version

mlflow, version 2.13.0


In [8]:
TAXI_DATA_DIR = "taxi_data"
os.makedirs(TAXI_DATA_DIR, exist_ok=True)

### Downloading data

In [17]:
link_taxi_jan = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet"
link_taxi_feb = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet"
link_taxi_mar = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet"

In [28]:
! wget {link_taxi_jan} -P {TAXI_DATA_DIR}
! wget {link_taxi_feb} -P {TAXI_DATA_DIR}
! wget {link_taxi_mar} -P {TAXI_DATA_DIR}

--2024-05-22 18:01:30--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.119, 18.239.238.152, 18.239.238.212, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.119|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: ‘taxi_data/green_tripdata_2023-01.parquet’

green_tripdata_2023 100%[===================>]   1.36M  --.-KB/s    in 0.02s   

2024-05-22 18:01:30 (74.6 MB/s) - ‘taxi_data/green_tripdata_2023-01.parquet’ saved [1427002/1427002]

--2024-05-22 18:01:30--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.119, 18.239.238.152, 18.239.238.212, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.119|:443...

In [30]:
! ls {TAXI_DATA_DIR}

green_tripdata_2023-01.parquet	green_tripdata_2023-03.parquet
green_tripdata_2023-02.parquet


### Running Preprocess script

In [32]:
! python homework/preprocess_data.py --raw_data_path {TAXI_DATA_DIR} --dest_path ./output

In [35]:
! ls output | wc -l

4


### Answer to Q2 : 4 files

## Q3: Train a model with autolog

In [54]:
from homework import train
from pathlib import Path

In [57]:
data_path = Path("./output")
os.listdir(data_path)

['val.pkl', 'test.pkl', 'dv.pkl', 'train.pkl']

In [58]:
with mlflow.start_run():
    train.run_train(str(data_path))

Usage: ipykernel_launcher.py [OPTIONS]
Try 'ipykernel_launcher.py --help' for help.

Error: Got unexpected extra arguments (o u t p u t)


AttributeError: 'tuple' object has no attribute 'tb_frame'